## QA over unstructured data

Using Match LSTM, Pointer Networks, as mentioned in paper https://arxiv.org/pdf/1608.07905.pdf

We start with the pre-processing provided by https://github.com/MurtyShikhar/Question-Answering to clean up the data and make neat para, ques files.


### @TODOs:

1. [done] _Figure out how to put in real, pre-trained embeddings in embeddings layer._
2. [done] _Explicitly provide batch size when instantiating model_
3. is ./val.ids.* validation set or test set?: **validation**
4. [done:em] emInstead of test loss, calculate test acc metrics
    1. todo: new metrics like P, R, F1
5. Update unit test codes

In [1]:
# # Codeblock to pull up embeddings. Needs to run before following imports
# import numpy as np

# # Macros 
# DATA_LOC = './data/squad/'
# EMBEDDING_FILE = 'glove.trimmed.300.npz'
# VOCAB_FILE = 'vocab.dat'

# file_loc = DATA_LOC + EMBEDDING_FILE
# glove_file = np.load(open(file_loc))['glove']

In [2]:
from __future__ import unicode_literals, print_function, division
import matplotlib.pyplot as plt
from io import open
import numpy as np
import unicodedata
import traceback
import string
import random
import time
import re
import os


import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable

device = torch.device("cuda")

torch.manual_seed(42)
np.random.seed(42)

#### Debug Legend

- 5: Print everything that goes in every tensor.
- 4: ??
- 3: Check every model individually
- 2: Print things in training loops
- 1: ??

In [3]:
# Macros 
DATA_LOC = './data/squad/'
DEBUG = 2

# nn Macros
QUES_LEN, PARA_LEN =  30, 200
VOCAB_SIZE = 120000
# VOCAB_SIZE = glove_file.shape[1]               # @TODO: get actual size
HIDDEN_DIM = 150
EMBEDDING_DIM = 300
BATCH_SIZE = 20                  # Might have total 100 batches.
EPOCHS = 300
TEST_EVERY_ = 1
LR = 0.001

### Encoder 
Use a simple lstm class to have encoder for question and paragraph. 
The output of these will be used in the match lstm

$H^p = LSTM(P)$ 


$H^q = LSTM(Q)$

In [4]:
class Encoder(nn.Module):
    
    def __init__(self, inputlen, macros, glove_file):
        super(Encoder, self).__init__()
        
        # Catch dim
        self.inputlen = inputlen
        self.hiddendim = macros['hidden_dim']
        self.embeddingdim =  macros['embedding_dim']
        self.vocablen = macros['vocab_size']
        
        self.batch_size = macros['batch_size']
        self.debug = macros['debug']
        
        # Embedding Layer
#         self.embedding = nn.Embedding(self.vocablen, self.embeddingdim)
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(glove_file))
        self.embedding.weight.requires_grad = True
       
        # LSTM Layer
        self.lstm = nn.LSTM(self.embeddingdim, self.hiddendim)
        
    def init_hidden(self, batch_size):
        
        # Returns a new hidden layer var for LSTM
        return (torch.zeros((1, batch_size, self.hiddendim), device=device), 
                torch.zeros((1, batch_size, self.hiddendim), device=device))
    
    def forward(self, x, h):
        
        # Input: x (batch, len ) (current input)
        # Hidden: h (1, batch, hiddendim) (last hidden state)
        
        # Batchsize: b int (inferred)
        b = x.shape[0]
        
        if self.debug > 4: print("x:\t", x.shape)
        if self.debug > 4: print("h:\t", h[0].shape, h[1].shape)
        
        x_emb = self.embedding(x)
        if self.debug > 4: print("x_emb:\t", x_emb.shape)
            
        ycap, h = self.lstm(x_emb.view(-1, b, self.embeddingdim), h)
        if self.debug > 4: print("ycap:\t", ycap.shape)
        
        return ycap, h
    
    
# with torch.no_grad():
#     print ("Trying out question encoder LSTM")
#     model = Encoder(QUES_LEN, HIDDEN_DIM, EMBEDDING_DIM, VOCAB_SIZE)
#     dummy_x = torch.tensor([22,45,12], dtype=torch.long)
#     hidden = model.init_hidden()
#     ycap, h = model(dummy_x, hidden)
    
#     print(ycap.shape)
#     print(h[0].shape, h[1].shape)


if DEBUG > 2:
    with torch.no_grad():

        dummy_para = torch.randint(0,VOCAB_SIZE-1,(PARA_LEN*BATCH_SIZE,), device=device).view(BATCH_SIZE,PARA_LEN).long()
    #     print (dummy_para.shape)
        dummy_question = torch.randint(0,VOCAB_SIZE-1,(QUES_LEN*BATCH_SIZE,), device=device).view(BATCH_SIZE,QUES_LEN).long()
    #     print (dummy_question.shape)

    #     print("LSTM with batches")
        ques_model = Encoder(QUES_LEN, HIDDEN_DIM, EMBEDDING_DIM, VOCAB_SIZE).cuda(device)
        para_model = Encoder(QUES_LEN, HIDDEN_DIM, EMBEDDING_DIM, VOCAB_SIZE).cuda(device)
        ques_hidden = ques_model.init_hidden()
        para_hidden = para_model.init_hidden()
        ques_embedded,hidden_ques = ques_model(dummy_question,ques_hidden)
        para_embedded,hidden_para = para_model(dummy_para,para_hidden)
        
        print (ques_embedded.shape) # question_length,batch,embedding_dim
        print (para_embedded.shape) # para_length,batch,embedding_dim
        print (hidden_para[0].shape,hidden_para[1].shape)

### Match LSTM

Use a match LSTM to compute a **summarized sequential vector** for the paragraph w.r.t the question.

Consider the summarized vector ($H^r$) as the output of a new decoder, where the inputs are $H^p, H^q$ computed above. 

1. Attend the para word $i$ with the entire question ($H^q$)
  
    1. $\vec{G}_i = tanh(W^qH^q + repeat(W^ph^p_i + W^r\vec{h^r_{i-1} + b^p}))$
    
    2. *Computing it*: Here, $\vec{G}_i$ is equivalent to `energy`, computed differently.
    
    3. Use a linear layer to compute the content within the $repeat$ fn.
    
    4. Add with another linear (without bias) with $H_q$
    
    5. $tanh$ the bloody thing
  
  
2. Softmax over it to get $\alpha$ weights.

    1. $\vec{\alpha_i} = softmax(w^t\vec{G}_i + repeat(b))$
    
3. Use the attention weight vector $\vec{\alpha_i}$ to obtain a weighted version of the question and concat it with the current token of the passage to form a vector $\vec{z_i}$

4. Use $\vec{z_i}$ to compute the desired $h^r_i$:

    1. $ h^r_i = LSTM(\vec{z_i}, h^r_{i-1}) $
    


In [5]:
class MatchLSTMEncoder(nn.Module):
    
    def __init__(self, macros):
        
        super(MatchLSTMEncoder, self).__init__()
        
        self.hidden_dim = macros['hidden_dim']
        self.ques_len = macros['ques_len']
        self.batch_size = macros['batch_size']
        self.debug = macros['debug']    
        
        # Catch lens and params
        self.lin_g_repeat = nn.Linear(2*self.hidden_dim, self.hidden_dim)
        self.lin_g_nobias = nn.Linear(self.hidden_dim, self.hidden_dim)
        
        self.alpha_i_w = nn.Parameter(torch.FloatTensor(self.hidden_dim, 1))
        self.alpha_i_b = nn.Parameter(torch.FloatTensor((1)))
        
        self.lstm_summary = nn.LSTM(self.hidden_dim*(self.ques_len+2), self.hidden_dim)
                                      
    
    def forward(self, H_p, h_ri, H_q, hidden):
        """
            Ideally, we would have manually unrolled the lstm 
            but due to memory constraints, we do it in the module.
        """
        
        # Find the batchsize
        batch_size = H_p.shape[1]
        
        H_r = torch.empty((0, batch_size, self.hidden_dim), device=device, dtype=torch.float)
        H_r = torch.cat((H_r, h_ri), dim=0)
        
        if self.debug > 4:
            print( "H_p:\t\t\t", H_p.shape)
            print( "h_ri:\t\t\t", h_ri.shape)
            print( "H_q:\t\t\t", H_q.shape)
        
        for i in range(H_p.shape[0]):
            
            lin_repeat_input = torch.cat((H_p[i].view(1, batch_size, -1), H_r[i].view(1, batch_size, -1)), dim=2)
            if self.debug > 4: print("lin_repeat_input:\t", lin_repeat_input.shape)

            lin_g_input_b = self.lin_g_repeat(lin_repeat_input)
            if self.debug > 4: print("lin_g_input_b unrepeated:", lin_g_input_b.shape)

            lin_g_input_b = lin_g_input_b.repeat(H_q.shape[0], 1, 1)
            if self.debug > 4: print("lin_g_input_b:\t\t", lin_g_input_b.shape)

            # lin_g_input_a = self.lin_g_nobias.matmul(H_q.view(-1, self.ques_len, self.hidden_dim)) #self.lin_g_nobias(H_q)
            lin_g_input_a =  self.lin_g_nobias(H_q)
            if self.debug > 4: print("lin_g_input_a:\t\t", lin_g_input_a.shape)

            G_i = F.tanh(lin_g_input_a + lin_g_input_b)
            if self.debug > 4: print("G_i:\t\t\t", G_i.shape)
            # Note; G_i should be a 1D vector over ques_len

            # Attention weights
            alpha_i_input_a = G_i.view(batch_size, -1, self.hidden_dim).matmul(self.alpha_i_w).view(batch_size, 1, -1)
            if self.debug > 4: print("alpha_i_input_a:\t", alpha_i_input_a.shape)

            alpha_i_input = alpha_i_input_a.add_(self.alpha_i_b.view(-1,1,1).repeat(1,1,self.ques_len))
            if self.debug > 4: print("alpha_i_input:\t\t", alpha_i_input.shape)

            # Softmax over alpha inputs
            alpha_i = F.softmax(alpha_i_input, dim=-1)
            if self.debug > 4: print("alpha_i:\t\t", alpha_i.shape)

            # Weighted summary of question with alpha    
            z_i_input_b = (
                            H_q.view(batch_size, self.ques_len, -1) *
                           (alpha_i.view(batch_size, self.ques_len, -1).repeat(1, 1, self.hidden_dim))
                          ).view(self.ques_len,batch_size, -1)
            if self.debug > 4: print("z_i_input_b:\t\t", z_i_input_b.shape)

            z_i = torch.cat((H_p[i].view(1, batch_size, -1), z_i_input_b), dim=0)
            if self.debug > 4: print("z_i:\t\t\t", z_i.shape)

            # Pass z_i, h_ri to the LSTM 
            lstm_input = torch.cat((z_i.view(1, batch_size,-1), H_r[i].view(1, batch_size, -1)), dim=2)
            if self.debug > 4: print("lstm_input:\t\t", lstm_input.shape)

            # Take input from LSTM, concat in H_r and nullify the temp var.
            h_ri, hidden = self.lstm_summary(lstm_input, hidden)
            H_r = torch.cat((H_r, h_ri), dim=0)
            h_ri = None
            
            if self.debug > 4:
                print("\tH_r:\t\t\t", H_r.shape)
#                 print("hidden new:\t\t", hidden[0].shape, hidden[1].shape)

        return H_r[1:]
    
    def init_hidden(self, batch_size):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros((1, batch_size, self.hidden_dim), device=device),
                torch.zeros((1, batch_size, self.hidden_dim), device=device))

# with torch.no_grad():
#     model = MatchLSTMEncoder(HIDDEN_DIM, QUES_LEN)
#     h_pi = torch.randn(1, BATCH_SIZE, HIDDEN_DIM)
#     h_ri = torch.randn(1, BATCH_SIZE, HIDDEN_DIM)
#     hidden = model.init_hidden()
#     H_q = torch.randn(QUES_LEN, BATCH_SIZE, HIDDEN_DIM)
    
#     op, hid = model(h_pi, h_ri, H_q, hidden)
    
#     print("\nDone:op", op.shape)
#     print("Done:hid", hid[0].shape, hid[1].shape)

if DEBUG > 2:
    with torch.no_grad():
        matchLSTMEncoder = MatchLSTMEncoder(HIDDEN_DIM, QUES_LEN).cuda(device)
        hidden = matchLSTMEncoder.init_hidden()
        para_embedded = torch.rand((PARA_LEN, BATCH_SIZE, HIDDEN_DIM), device=device)
        ques_embedded = torch.rand((QUES_LEN, BATCH_SIZE, HIDDEN_DIM), device=device)
        h_ri = torch.randn(1, BATCH_SIZE, HIDDEN_DIM, device=device)
    #     if DEBUG:
    #         print ("init h_ri shape is: ", h_ri.shape)
    #         print ("the para length is ", len(para_embedded))
        H_r = matchLSTMEncoder(para_embedded.view(-1,BATCH_SIZE,HIDDEN_DIM),
                               h_ri, 
                               ques_embedded, 
                               hidden)
        print("H_r: ", H_r.shape)
        
        
        

### Pointer Network

Using a ptrnet over $H_r$ to unfold and get most probable spans.
We use the **boundry model** to do that (predict start and end of seq).

A simple energy -> softmax -> decoder. Where softmaxed energy is supervised.

In [6]:
class PointerDecoder(nn.Module):
    
    def __init__(self, macros):
        super(PointerDecoder, self).__init__()
        
        # Keep args
        self.hidden_dim = macros['hidden_dim']
        self.batch_size = macros['batch_size']
        self.para_len = macros['para_len']
        self.debug = macros['debug']
        
        self.lin_f_repeat = nn.Linear(self.hidden_dim, self.hidden_dim)
        self.lin_f_nobias = nn.Linear(self.hidden_dim, self.hidden_dim, bias=False)
        
        self.beta_k_w = nn.Parameter(torch.FloatTensor(self.hidden_dim, 1))
        self.beta_k_b = nn.Parameter(torch.FloatTensor(1))
        
        self.lstm = nn.LSTM(self.hidden_dim*(PARA_LEN+1), self.hidden_dim)

    
    def init_hidden(self, batch_size):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros((1, batch_size, self.hidden_dim), device=device),
                torch.zeros((1, batch_size, self.hidden_dim), device=device))
    
    def forward(self, h_ak, H_r, hidden):
        
        # h_ak (current decoder's last op) (1,batch,hiddendim)
        # H_r (weighted summary of para) (P, batch, hiddendim)
        batch_size = H_r.shape[1]
        
        if self.debug > 4:
            print("h_ak:\t\t\t", h_ak.shape)
            print("H_r:\t\t\t", H_r.shape)
            print("hidden:\t\t\t", hidden[0].shape, hidden[1].shape)
            
        # Prepare inputs for the tanh used to compute energy
        f_input_b = self.lin_f_repeat(h_ak)
        if self.debug > 4: print("f_input_b unrepeated:  ", f_input_b.shape)
        
        #H_r shape is ([PARA_LEN, BATCHSIZE, EmbeddingDIM])
        f_input_b = f_input_b.repeat(H_r.shape[0], 1, 1)
        if self.debug > 4: print("f_input_b repeated:\t", f_input_b.shape)
            
        f_input_a = self.lin_f_nobias(H_r)
        if self.debug > 4: print("f_input_a:\t\t", f_input_a.shape)
            
        # Send it off to tanh now
        F_k = F.tanh(f_input_a+f_input_b)
        if self.debug > 4: print("F_k:\t\t\t", F_k.shape) #PARA_LEN,BATCHSIZE,EmbeddingDim
            
        # Attention weights
        beta_k_input_a = F_k.view(batch_size, -1, self.hidden_dim).matmul(self.beta_k_w).view(batch_size, 1, -1)
        if self.debug > 4: print("beta_k_input_a:\t\t", beta_k_input_a.shape)
            
        beta_k_input = beta_k_input_a.add_(self.beta_k_b.repeat(1,1,self.para_len))
        if self.debug > 4: print("beta_k_input:\t\t", beta_k_input.shape)
            
        beta_k = F.softmax(beta_k_input, dim=-1)
        if self.debug > 4: print("beta_k:\t\t\t", beta_k.shape)
            
        lstm_input_a = H_r.view(batch_size, self.para_len, -1) * (beta_k.view(batch_size, self.para_len, -1).repeat(1,1,self.hidden_dim))
        if self.debug > 4: print("lstm_input_a:\t\t", lstm_input_a.shape)
            
        lstm_input = torch.cat((lstm_input_a.view(1, batch_size,-1), h_ak.view(1, batch_size, -1)), dim=2)
        if self.debug > 4: print("lstm_input:\t\t", lstm_input.shape)
        
        h_ak, hidden = self.lstm(lstm_input, hidden)
        
        return h_ak, hidden, beta_k
            
if DEBUG > 2:
    with torch.no_grad():
        pointerDecoder = PointerDecoder(HIDDEN_DIM).cuda(device)
        h_ak = torch.randn(1,BATCH_SIZE,HIDDEN_DIM, device=device)
    #     H_r = torch.randn(PARA_LEN, BATCH_SIZE, HIDDEN_DIM)
        pointerHidden = pointerDecoder.init_hidden()
        h_ak, hidden, beta_k = pointerDecoder(h_ak, para_embedded, hidden)
        print (beta_k.shape)

# Pull the real data from disk.

Files stored in `./data/squad/train.ids.*`
Pull both train and test.

In [7]:
def prepare_data(data_loc, macros, crop=None):
    """
        Given the dataloc and the data available in a specific format, it would pick the data up, and make trainable matrices,
        Harvest train_P, train_Q, train_Y, test_P, test_Q, test_Y matrices in this format
        
        If crop given, will trim the data at a certain length
        
        **return_type**: np matrices
    """
    
    # Unpacking macros
    PARA_LEN = macros['para_len']
    QUES_LEN = macros['ques_len']
    
    train_q = np.asarray([[int(x) for x in datum.split()] for datum in list(open(os.path.join(data_loc, 'train.ids.question')))])
    train_p = np.asarray([[int(x) for x in datum.split()] for datum in list(open(os.path.join(data_loc, 'train.ids.context')))])
    train_y = np.asarray([[int(x) for x in datum.split()] for datum in list(open(os.path.join(data_loc, 'train.span')))])

    test_q = np.asarray([[int(x) for x in datum.split()] for datum in list(open(os.path.join(data_loc, 'val.ids.question')))])
    test_p = np.asarray([[int(x) for x in datum.split()] for datum in list(open(os.path.join(data_loc, 'val.ids.context')))])
    test_y = np.asarray([[int(x) for x in datum.split()] for datum in list(open(os.path.join(data_loc, 'val.span')))])

    if macros['debug'] > 3:
        print("Train Q: ", train_q.shape)
        print("Train P: ", train_p.shape)
        print("Train Y: ", train_y.shape)
        print("Test Q: ", test_q.shape)
        print("Test P: ", test_p.shape)
        print("Test Y: ", test_y.shape)
    
    """
        Parse the semi-raw data:
            - shuffle
            - pad, prepare
            - dump useless vars
    """
    # Shuffle data
    
    if crop:
        index_train, index_test = np.random.choice(np.arange(len(train_p)), crop), \
                                  np.random.choice(np.arange(len(test_p)), crop)
    else:
        index_train, index_test = np.arange(len(train_p)), np.arange(len(test_p))
        np.random.shuffle(index_train)
        np.random.shuffle(index_test)

    train_p, train_q, train_y = train_p[index_train], train_q[index_train], train_y[index_train]
    test_p, test_q, test_y = test_p[index_test], test_q[index_test], test_y[index_test]

#     sanity_check(train_p, train_y)
    
    # Pad and prepare
    train_P = np.zeros((len(train_p), PARA_LEN))
    train_Q = np.zeros((len(train_q), QUES_LEN))
    train_Y_start = np.zeros((len(train_p), PARA_LEN))
    train_Y_end = np.zeros((len(train_p), PARA_LEN))

    test_P = np.zeros((len(test_p), PARA_LEN))
    test_Q = np.zeros((len(test_q), QUES_LEN))
    test_Y_start = np.zeros((len(test_p), PARA_LEN))
    test_Y_end = np.zeros((len(test_p), PARA_LEN))
    
#     print(train_P.shape)

    crop_train = []    # Remove these rows from training
    for i in range(len(train_p)):
        p = train_p[i]
        q = train_q[i]
        y = train_y[i]

        # First see if you can keep this example or not (due to size)
        if y[0] >= PARA_LEN or y[1] >= PARA_LEN:
            crop_train.append(i)
            continue


        train_P[i, :min(PARA_LEN, len(p))] = p[:min(PARA_LEN, len(p))]
        train_Q[i, :min(QUES_LEN, len(q))] = p[:min(QUES_LEN, len(q))]
        train_Y_start[i, y[0]] = 1
        train_Y_end[i, y[1]] = 1

    crop_test = []
    for i in range(len(test_p)):
        p = test_p[i]
        q = test_q[i]
        y = test_y[i]

        # First see if you can keep this example or not (due to size)
        if y[0] >= PARA_LEN or y[1] >= PARA_LEN:
            crop_test.append(i)
            continue

        test_P[i, :min(PARA_LEN, len(p))] = p[:min(PARA_LEN, len(p))]
        test_Q[i, :min(QUES_LEN, len(q))] = p[:min(QUES_LEN, len(q))]
        test_Y_start[i, y[0]] = 1
        test_Y_end[i, y[1]] = 1
        
        
    # Remove the instances which are in crop_train
    train_P = np.delete(train_P, crop_train, axis=0)
    train_Q = np.delete(train_Q, crop_train, axis=0)
    train_Y_start = np.delete(train_Y_start, crop_train, axis=0)
    train_Y_end = np.delete(train_Y_end, crop_train, axis=0)
    
    test_P = np.delete(test_P, crop_test, axis=0)
    test_Q = np.delete(test_Q, crop_test, axis=0)
    test_Y_start = np.delete(test_Y_start, crop_test, axis=0)
    test_Y_end = np.delete(test_Y_end, crop_test, axis=0)

    if macros['debug'] >= 1:
        print("Train Q: ", train_Q.shape)
        print("Train P: ", train_P.shape)
        print("Train Y: ", train_Y_start.shape)
        print("Test Q: ", test_Q.shape)
        print("Test P: ", test_P.shape)
        print("Test Y: ", test_Y_start.shape)
        print("Crop_train: ", len(crop_train))
        print("Crop_test: ", len(crop_test))
    # Let's free up some memory now
    train_p, train_q, train_y, test_p, test_q, test_y = None, None, None, None, None, None
    
    # Load embedding matrics
    vectors = np.load(os.path.join(data_loc, 'glove.trimmed.300.npz'))['glove']
    
    return train_P, train_Q, train_Y_start, train_Y_end, test_P, test_Q, test_Y_start, test_Y_end, vectors

In [8]:
# macros = {
#     "ques_len": QUES_LEN,
#     "hidden_dim": HIDDEN_DIM, 
#     "vocab_size": VOCAB_SIZE, 
#     "batch_size": BATCH_SIZE,
#     "para_len": PARA_LEN,
#     "embedding_dim": EMBEDDING_DIM,
#     "debug": 4
# } 

# a = prepare_data(DATA_LOC, macros=macros, crop=200)

# Training, and running the model
- Write a train fn
- Write a training loop invoking it
- Fill in real data

----------

Feats:
- Function to test every n epochs.
- Report train accuracy every epoch
- Store the train, test accuracy for every instance.


In [9]:
def train(para_batch,
          ques_batch,
          answer_start_batch,
          answer_end_batch,
          ques_model,
          para_model,
          match_LSTM_encoder_model,
          pointer_decoder_model,
          optimizer, 
          loss_fn,
          macros,
          debug=2):

    """
    
    :param para_batch: paragraphs (batch, max_seq_len_para) 
    :param ques_batch: questions corresponding to para (batch, max_seq_len_ques)
    :param answer_start_batch: one-hot vector denoting pos of span start (batch, max_seq_len_para)
    :param answer_end_batch: one-hot vector denoting pos of span end (batch, max_seq_len_para)
    
    # Models
    :param ques_model: model to encode ques
    :param para_model: model to encode para
    :param match_LSTM_encoder_model: model to match para, ques to get para summary
    :param pointer_decoder_model: model to get a pointer over start and end span pointer
    
    # Loss and Optimizer.
    :param loss_fn: 
    :param optimizer: 
    
    :return: 
    
    
    NOTE: When using MSE, 
        - target labels are one-hot
        - target label is float tensor
        - shape (batch, 1, len)
        
        When using CrossEntropy
        - target is not onehot
        - long
        - shape (batch, )
    """
    try:    
    #     DEBUG = debug
    #     BATCH_SIZE = macros['batch_size']
    #     HIDDEN_DIM = macros['hidden_dim']

        if debug >=2: 
            print("\tpara_batch:\t\t", para_batch.shape)
            print("\tques_batch:\t\t", ques_batch.shape)
            print("\tanswer_start_batch:\t", answer_start_batch.shape)
            print("\tanswer_end_batch:\t\t", answer_end_batch.shape)

        # Wiping all gradients
        optimizer.zero_grad()

        # Initializing all hidden states.
        hidden_quesenc = ques_model.init_hidden(macros['batch_size'])
        hidden_paraenc = para_model.init_hidden(macros['batch_size'])
        hidden_mlstm = match_LSTM_encoder_model.init_hidden(macros['batch_size'])
        hidden_ptrnet = pointer_decoder_model.init_hidden(macros['batch_size'])
        h_ri = torch.zeros((1, macros['batch_size'], macros['hidden_dim']), dtype=torch.float, device=device)
        h_ak = torch.zeros((1, macros['batch_size'], macros['hidden_dim']), dtype=torch.float, device=device)
        if debug >= 2: print("------------Instantiated hidden states------------")

        #passing the data through LSTM pre-processing layer
        H_q, ques_model_hidden = ques_model(ques_batch, hidden_quesenc)
        H_p, para_model_hidden = para_model(para_batch, hidden_paraenc)
        if debug >= 2: 
            print("\tH_q:\t\t", H_q.shape)
            print("\tH_p:\t\t", H_p.shape)
            print("\tH_ri:\t\t", h_ri.shape)
    #         raw_input("Check memory and ye shall continue")
            print("------------Encoded hidden states------------")

        H_r = match_LSTM_encoder_model(H_p.view(-1, macros['batch_size'], macros['hidden_dim']), h_ri, H_q, hidden_mlstm)
        if debug >= 2: print("------------Passed through matchlstm------------")

        #Passing the paragraph embddin via pointer network to generate final answer pointer.
        h_ak, hidden_ptrnet, beta_k_start = pointer_decoder_model(h_ak, H_r, hidden_ptrnet)
        h_ak, hidden_ptrnet, beta_k_end = pointer_decoder_model(h_ak, H_r, hidden_ptrnet)
        if debug >= 2: print("------------Passed through pointernet------------")


        # For crossentropy
        _, answer_start_batch = answer_start_batch.max(dim=2)
        _, answer_end_batch = answer_end_batch.max(dim=2)
        answer_start_batch = answer_start_batch.view(-1).long()
        answer_end_batch = answer_end_batch.view(-1).long()
#         print(beta_k_start.view(-1, macros['para_len']).shape, answer_start_batch.view(-1).shape)
        
        # Calculate Loss
        loss = loss_fn(beta_k_start.view(-1, macros['para_len']), answer_start_batch)
        loss += loss_fn(beta_k_end.view(-1, macros['para_len']), answer_end_batch)
#         loss = loss_fn(beta_k_start, answer_start_batch)
#         loss += loss_fn(beta_k_end, answer_end_batch)
        if debug >= 2: print("------------Calculated loss------------")

        loss.backward()
        if debug >= 2: print("------------Calculated Gradients------------")

        #optimization step
        optimizer.step()
        if debug >= 2: print("------------Updated weights.------------")
            
        return beta_k_start, beta_k_end, loss
    
    except: 
        traceback.print_exc()

In [10]:
# Predict function (no grad, no eval)
def predict(para_batch,
            ques_batch,
            ques_model,
            para_model,
            match_LSTM_encoder_model,
            pointer_decoder_model,
            macros,
            loss_fn,
            debug):
    """
        Function which returns the model's output based on a given set of P&Q's. 
        Does not convert to strings, gives the direct model output.
        
        Expects:
            four models
            data
            misc macros
    """
    
#     BATCH_SIZE = macros['batch_size']
    BATCH_SIZE = ques_batch.shape[0]
    HIDDEN_DIM = macros['hidden_dim']
    DEBUG = debug
    
    if debug >=2: 
        print("\tpara_batch:\t\t", para_batch.shape)
        print("\tques_batch:\t\t", ques_batch.shape)
        
    with torch.no_grad():    

        # Initializing all hidden states.
        hidden_quesenc = ques_model.init_hidden(BATCH_SIZE)
        hidden_paraenc = para_model.init_hidden(BATCH_SIZE)
        hidden_mlstm = match_LSTM_encoder_model.init_hidden(BATCH_SIZE)
        hidden_ptrnet = pointer_decoder_model.init_hidden(BATCH_SIZE)
        h_ri = torch.zeros((1, BATCH_SIZE, HIDDEN_DIM), dtype=torch.float, device=device)
        h_ak = torch.zeros((1, BATCH_SIZE, HIDDEN_DIM), dtype=torch.float, device=device)
        if DEBUG >= 2: print("------------Instantiated hidden states------------")
            
        #passing the data through LSTM pre-processing layer
        H_q, ques_model_hidden = ques_model(ques_batch, hidden_quesenc)
        H_p, para_model_hidden = para_model(para_batch, hidden_paraenc)
        if DEBUG >= 2: 
            print("\tH_q:\t\t", H_q.shape)
            print("\tH_p:\t\t", H_p.shape)
            print("\tH_ri:\t\t", h_ri.shape)
#             raw_input("Check memory and ye shall continue")
            print("------------Encoded hidden states------------")

        H_r = match_LSTM_encoder_model(H_p.view(-1, BATCH_SIZE, HIDDEN_DIM), h_ri, H_q, hidden_mlstm)
        if DEBUG >= 2: print("------------Passed through matchlstm------------")

        #Passing the paragraph embddin via pointer network to generate final answer pointer.
        h_ak, hidden_ptrnet, beta_k_start = pointer_decoder_model(h_ak, H_r, hidden_ptrnet)
        _, _, beta_k_end = pointer_decoder_model(h_ak, H_r, hidden_ptrnet)
        if DEBUG >= 2: print("------------Passed through pointernet------------")
            
        loss = 0.0
                            
        # For crossentropy
#         _, answer_start_batch = answer_start_batch.max(dim=2)[1]
#         _, answer_end_batch = answer_end_batch.max(dim=2)[1]
#         print("labels: ", answer_start_batch.shape)[1]
            
#         #How will we manage batches for loss.
#         loss = loss_fn(beta_k_start, answer_start_batch)
#         loss += loss_fn(beta_k_end, answer_end_batch)
#         if debug >= 2: print("------------Calculated loss------------")
            
        return (beta_k_start, beta_k_end, loss)


In [11]:
# Eval function (no grad no eval no nothing)
def eval(y_cap, y, metrics={'em':None}):
    """ 
        Returns the exact-match (em) metric by default.
        Can specifiy more in a list (TODO)
        
        Inputs:
        - y_cap: list of two tensors (start, end) of dim [BATCH_SIZE, PARA_LEN] each
        - y: list of two tensors (start, end) of dim [BATCH_SIZE, 1] each
    """
    
#     print(y[0].shape, y[1].shape, y_cap[0].shape, y_cap[1].shape)
    
    y_cap= torch.argmax(y_cap[0], dim=1).float(), torch.argmax(y_cap[1], dim=1).float()
    y = torch.argmax(y[0], dim=1).float(), torch.argmax(y[1], dim=1).float()
    
    if "em" in metrics.keys():
        metrics['em'] = (y[0].eq(y_cap[0]) & y[1].eq(y_cap[1])).sum().item()/ float(y[0].shape[0])
        
    if DEBUG >= 3: 
        print("Test performance: ", metrics)
        print("------------Evaluated------------")
        
    return metrics

if DEBUG >=5:
    # Testing this function
    metrics = {'em':None}
#     y = torch.tensor([[3]]).float(), torch.tensor([[4]]).float()
    y = torch.tensor([[0,0,3,0], [0,2,0,0]]), torch.tensor([[0,0,0,3], [0,0,0,3]])
    y_cap = torch.tensor([[0,0,3,0],[0,0,3,0]]), torch.tensor([[0,0,0,3],[0,0,0,3]])
#     y = torch.randint(0, PARA_LEN, (BATCH_SIZE,)).float(), torch.randint(0, PARA_LEN, (BATCH_SIZE,)).float()
#     y_cap = torch.rand((BATCH_SIZE, PARA_LEN)), torch.rand((BATCH_SIZE, PARA_LEN))
    print(eval(y_cap, y))   

In [12]:
def training_loop(_models, _data, _macros, _epochs, _save_best=False, _test_eval=0, _train_eval=0, _debug=2):
    """
        > Instantiate models
        > Instantiate loss, optimizer
        > Instantiate ways to store loss

        > Per epoch
            > sample batch and give to train fn
            > get loss
            > if epoch %k ==0: get test accuracy

        > have fn to calculate test accuracy
    """
    try: 
        # Unpack data
        DEBUG = _debug
        train_P = _data['train']['P']
        train_Q = _data['train']['Q']
        train_Y_start = _data['train']['Ys']
        train_Y_end = _data['train']['Ye']
        test_P = _data['test']['P']
        test_Q = _data['test']['Q']
        test_Y_start = _data['test']['Ys']
        test_Y_end = _data['test']['Ye']

        ques_model, para_model, match_LSTM_encoder_model, pointer_decoder_model = _models
        _data = None

        # Instantiate Loss
#         loss_fn = nn.MSELoss()
        loss_fn = nn.CrossEntropyLoss()
        optimizer = optim.Adam(list(filter(lambda p: p.requires_grad, ques_model.parameters())) + 
                                 list(filter(lambda p: p.requires_grad, para_model.parameters())) + 
                                 list(match_LSTM_encoder_model.parameters()) + 
                                 list(pointer_decoder_model.parameters()), lr=macros['lr'])
#         optimizer = optim.Adam(list(ques_model.parameters()) + \
#                                list(para_model.parameters()) + \
#                                list(match_LSTM_encoder_model.parameters()) + \
#                               list(pointer_decoder_model.parameters()), lr=macros['lr'])

        # Losses
        train_losses = []
        train_em = []
        test_losses = []
        test_em = []

        # Training Loop
        for epoch in range(_epochs):
            print("Epoch: ", epoch, "/", _epochs)

            epoch_loss = []
            epoch_train_em = []
            epoch_time = time.time()

            for iter in range(int(len(train_P)/BATCH_SIZE)):
    #         for iter in range(2):

                batch_time = time.time()

                # Sample batch and train on it
                sample_index = np.random.randint(0, len(train_P), _macros['batch_size'])
            
#                 grad_old = sum([x.grad.sum().item() for x in params])

                y_cap_start, y_cap_end, loss = train(
                    para_batch = torch.tensor(train_P[sample_index], dtype=torch.long, device=device),
                    ques_batch = torch.tensor(train_Q[sample_index], dtype=torch.long, device=device),
                    answer_start_batch = torch.tensor(train_Y_start[sample_index], dtype=torch.float, device=device).view( _macros['batch_size'], 1, _macros['para_len']),
                    answer_end_batch = torch.tensor(train_Y_end[sample_index], dtype=torch.float, device=device).view(_macros['batch_size'], 1, _macros['para_len']),
                    ques_model = ques_model,
                    para_model = para_model,
                    match_LSTM_encoder_model = match_LSTM_encoder_model,
                    pointer_decoder_model = pointer_decoder_model,
                    optimizer = optimizer, 
                    loss_fn= loss_fn,
                    macros=_macros,
                    debug=_macros['debug']
                )

                if _train_eval: 
                    # Calculate train accuracy for this minibatch
                    metrics = eval(
                        y=(torch.tensor(train_Y_start[sample_index], dtype=torch.long, device=device).view( -1, _macros['para_len']),
                            torch.tensor(train_Y_end[sample_index], dtype=torch.long, device=device).view(-1, _macros['para_len'])),
                        y_cap=[y_cap_start.squeeze(), y_cap_end.squeeze()])

                    epoch_train_em.append(metrics['em'])
    
                epoch_loss.append(loss.item())
    
#                 grad_new = sum([x.grad.sum().item() for x in params])

                print("Batch:\t%d" % iter,"/%d\t: " % (len(train_P)/_macros['batch_size']),
                      str("%s" % (time.time() - batch_time))[:8], 
                      str("\t\b%s" % (time.time() - epoch_time))[:10], 
                      "\tl:%f" % loss.item(),
                      "\tem:%f" % epoch_train_em[-1] if _train_eval else "")
#                      "\t\b\b%s" % grad_new - grad_old)
#                      end=None if iter+1 == int(len(train_P)/BATCH_SIZE) else "\r")

            train_losses.append(epoch_loss)
        
            if _train_eval: train_em.append(epoch_train_em)
            if _test_eval and epoch % _test_eval == 0:

                y_cap_start, y_cap_end, test_loss = predict(
                    para_batch = torch.tensor(test_P, dtype=torch.long, device=device),
                    ques_batch = torch.tensor(test_Q, dtype=torch.long, device=device),
                    ques_model = ques_model,
                    para_model = para_model,
                    match_LSTM_encoder_model = match_LSTM_encoder_model,
                    pointer_decoder_model = pointer_decoder_model,
                    macros = _macros,
                    loss_fn= loss_fn,
                    debug = _macros['debug']
                )
                metrics = eval(
                    y=(torch.tensor(test_Y_start, dtype=torch.long, device=device).view( -1, _macros['para_len']),
                        torch.tensor(test_Y_end, dtype=torch.long, device=device).view(-1, _macros['para_len'])),
                    y_cap=[y_cap_start.squeeze(), y_cap_end.squeeze()])

                test_losses.append(test_loss)
                test_em.append(metrics['em'])

            # At the end of every epoch, do print the average epoch loss, and other stat
            print("\nEpoch performance: ",
                  "%ssec" % str(time.time() - epoch_time)[:6],
                  "Trl:%f" % np.mean(epoch_loss, axis=0),
                  "\tTrem:%f" % np.mean(epoch_train_em) if _train_eval and epoch % _train_eval == 0 else "",
                  "\tTeem:%f\n" % test_em[-1] if _test_eval and epoch % _test_eval == 0 else "\n")

#         return train_losses, train_em, test_losses, test_em
    
    except KeyboardInterrupt:
        # someone called a ctrl+c on it. Let' return the things computed so far atlest.
        print("Found keyboard interrup")
    finally:
        return train_losses, train_em, test_losses, test_em
            
            

In [13]:
def visualize_loss(loss, _label="Some label", _only_epoch=True):
    """
        Fn to visualize loss.
        Expects either
            - [int, int] for epoch level stuff
            - [ [int, int], [int, int] ] for batch level data. 
    """
    
    plt.rcParams['figure.figsize'] = [15, 8] 
    
    # Detect input format
    if type(loss[0]) in [int, float, long]:
        
#         print("here")
        
        plt.plot(loss)
        plt.ylabel(_label)
        plt.show()
        
    elif type(loss[0]) == list:
        
        if _only_epoch:
            loss = [ sum(x) for x in loss ]
            
        else:
            loss = [ y for x in loss for y in x ]
            
        plt.plot(loss)
        plt.ylabel(_label)
        plt.show()        

## Orchestrator

One cell which instantiates and runs everything

In [14]:
"""
    Cell which pulls everything together.

    > init models
    > get data prepared
    > pass models and data to training loop
    > gets trained models and loss
    > saves models
    > visualizes loss?

No other function but this one ever sees global macros!
"""
macros = {
    "ques_len": QUES_LEN,
    "hidden_dim": HIDDEN_DIM, 
    "vocab_size": VOCAB_SIZE, 
    "batch_size": BATCH_SIZE,
    "para_len": PARA_LEN,
    "embedding_dim": EMBEDDING_DIM,
    "lr": LR,
    "debug":1
} 

data = {'train':{}, 'test':{}}
data['train']['P'], data['train']['Q'], data['train']['Ys'], data['train']['Ye'], \
data['test']['P'], data['test']['Q'], data['test']['Ys'], data['test']['Ye'], vectors = \
    prepare_data(DATA_LOC, macros, crop=200)

Train Q:  (197, 30)
Train P:  (197, 200)
Train Y:  (197, 200)
Test Q:  (196, 30)
Test P:  (196, 200)
Test Y:  (196, 200)
Crop_train:  3
Crop_test:  4


In [15]:
# # Instantiate modelshttp://localhost:8888/notebooks/model.ipynb#
ques_model = Encoder(QUES_LEN, macros, vectors).cuda(device)
para_model = Encoder(PARA_LEN, macros, vectors).cuda(device)
# ques_model = Encoder(QUES_LEN, macros, glove_file).cuda(device)
# para_model = Encoder(PARA_LEN, macros, glove_file).cuda(device)
match_LSTM_encoder_model = MatchLSTMEncoder(macros).cuda(device)
pointer_decoder_model = PointerDecoder(macros).cuda(device)

In [ ]:
op = training_loop(_models=[ques_model, para_model, match_LSTM_encoder_model, pointer_decoder_model],
                   _data=data,
                   _debug=macros['debug'],
                   _save_best=True,
                   _test_eval=TEST_EVERY_,
                       _train_eval=1,
#                        _test_every=0,
                      _epochs=EPOCHS,
                      _macros=macros)    

Epoch:  0 / 300
Batch:	0 /9	:  2.818527 2.818559 	l:10.596635 	em:0.000000
Batch:	1 /9	:  0.983757 3.803176 	l:10.596634 	em:0.000000
Batch:	2 /9	:  0.872685 4.677175 	l:10.596633 	em:0.000000
Batch:	3 /9	:  0.930788 5.608595 	l:10.596628 	em:0.050000
Batch:	4 /9	:  0.975600 6.584432 	l:10.596630 	em:0.000000
Batch:	5 /9	:  0.898082 7.483011 	l:10.596640 	em:0.000000
Batch:	6 /9	:  1.006263 8.490586 	l:10.596645 	em:0.000000
Batch:	7 /9	:  0.979976 9.470842 	l:10.596635 	em:0.000000
Batch:	8 /9	:  0.994686 10.46610 	l:10.596626 	em:0.000000

Epoch performance:  10.673sec Trl:10.596634 	Trem:0.005556 	Teem:0.000000

Epoch:  1 / 300
Batch:	0 /9	:  0.783880 0.783897 	l:10.596622 	em:0.000000
Batch:	1 /9	:  0.814975 1.599053 	l:10.596626 	em:0.000000
Batch:	2 /9	:  0.859468 2.459730 	l:10.596618 	em:0.000000
Batch:	3 /9	:  0.975922 3.436666 	l:10.596630 	em:0.000000
Batch:	4 /9	:  0.855581 4.293632 	l:10.596588 	em:0.000000
Batch:	5 /9	:  0.884494 5.179157 	l:10.596611 	em:0.000000
Batch:	

Batch:	2 /9	:  1.100679 3.040342 	l:10.548409 	em:0.000000
Batch:	3 /9	:  1.005305 4.046427 	l:10.593116 	em:0.000000
Batch:	4 /9	:  0.901406 4.948894 	l:10.539362 	em:0.000000
Batch:	5 /9	:  0.867437 5.817045 	l:10.495605 	em:0.000000
Batch:	6 /9	:  1.045692 6.864530 	l:10.486691 	em:0.000000
Batch:	7 /9	:  0.858674 7.723875 	l:10.542745 	em:0.000000
Batch:	8 /9	:  0.854707 8.579676 	l:10.429316 	em:0.000000

Epoch performance:  8.7814sec Trl:10.514480 	Trem:0.005556 	Teem:0.000000

Epoch:  14 / 300
Batch:	0 /9	:  1.052320 1.052338 	l:10.560362 	em:0.050000
Batch:	1 /9	:  0.865715 1.918521 	l:10.485687 	em:0.100000
Batch:	2 /9	:  0.873708 2.793241 	l:10.483668 	em:0.050000
Batch:	3 /9	:  0.906761 3.701382 	l:10.216731 	em:0.150000
Batch:	4 /9	:  0.932715 4.635413 	l:10.353626 	em:0.050000
Batch:	5 /9	:  1.090069 5.726673 	l:10.437686 	em:0.100000
Batch:	6 /9	:  0.894393 6.622340 	l:10.543930 	em:0.000000
Batch:	7 /9	:  1.278126 7.901498 	l:10.443743 	em:0.000000
Batch:	8 /9	:  1.10776

Batch:	4 /9	:  0.988896 4.832195 	l:9.955313 	em:0.100000
Batch:	5 /9	:  0.871320 5.704482 	l:10.167356 	em:0.150000
Batch:	6 /9	:  1.022459 6.727951 	l:10.066646 	em:0.200000
Batch:	7 /9	:  0.948024 7.676738 	l:9.792419 	em:0.150000
Batch:	8 /9	:  1.025294 8.702863 	l:9.943556 	em:0.200000

Epoch performance:  8.9101sec Trl:10.052020 	Trem:0.133333 	Teem:0.005102

Epoch:  27 / 300
Batch:	0 /9	:  1.106229 1.106266 	l:10.051878 	em:0.100000
Batch:	1 /9	:  1.004745 2.112704 	l:10.020861 	em:0.100000
Batch:	2 /9	:  1.102099 3.218211 	l:9.933932 	em:0.100000
Batch:	3 /9	:  0.860085 4.079150 	l:10.124531 	em:0.100000
Batch:	4 /9	:  0.901213 4.981445 	l:10.105121 	em:0.100000
Batch:	5 /9	:  1.152690 6.135529 	l:10.041957 	em:0.050000
Batch:	6 /9	:  1.050550 7.187495 	l:10.113184 	em:0.100000
Batch:	7 /9	:  0.825277 8.013928 	l:10.262756 	em:0.000000
Batch:	8 /9	:  0.940881 8.955470 	l:10.262173 	em:0.100000

Epoch performance:  9.1308sec Trl:10.101821 	Trem:0.083333 	Teem:0.000000

Epoch:  2

Batch:	7 /9	:  1.106374 8.082388 	l:9.606195 	em:0.300000
Batch:	8 /9	:  0.867517 8.950722 	l:10.033813 	em:0.100000

Epoch performance:  9.1622sec Trl:9.893066 	Trem:0.177778 	Teem:0.000000

Epoch:  40 / 300
Batch:	0 /9	:  1.030234 1.030256 	l:10.266333 	em:0.100000
Batch:	1 /9	:  1.127676 2.158873 	l:9.719887 	em:0.250000
Batch:	2 /9	:  1.057253 3.217629 	l:9.715950 	em:0.350000
Batch:	3 /9	:  1.037245 4.255633 	l:9.812652 	em:0.150000
Batch:	4 /9	:  0.913045 5.169729 	l:9.761528 	em:0.300000
Batch:	5 /9	:  0.979473 6.150536 	l:9.863295 	em:0.150000
Batch:	6 /9	:  0.999301 7.150414 	l:10.113524 	em:0.050000
Batch:	7 /9	:  0.914546 8.065405 	l:10.112679 	em:0.100000
Batch:	8 /9	:  1.026365 9.092551 	l:9.714720 	em:0.200000

Epoch performance:  9.3268sec Trl:9.897841 	Trem:0.183333 	Teem:0.000000

Epoch:  41 / 300
Batch:	0 /9	:  1.024530 1.024548 	l:9.560638 	em:0.300000
Batch:	1 /9	:  0.907291 1.932854 	l:9.864292 	em:0.200000
Batch:	2 /9	:  0.914049 2.849305 	l:9.763662 	em:0.250000


Batch:	0 /9	:  0.849961 0.849978 	l:9.819954 	em:0.150000
Batch:	1 /9	:  1.023099 1.874281 	l:9.813505 	em:0.150000
Batch:	2 /9	:  1.062875 2.940212 	l:9.833814 	em:0.100000
Batch:	3 /9	:  0.864665 3.806195 	l:9.663590 	em:0.200000
Batch:	4 /9	:  0.854906 4.662353 	l:9.663122 	em:0.250000
Batch:	5 /9	:  0.911199 5.573781 	l:10.036688 	em:0.150000
Batch:	6 /9	:  0.930143 6.505167 	l:9.949518 	em:0.150000
Batch:	7 /9	:  0.861891 7.368335 	l:10.093718 	em:0.100000
Batch:	8 /9	:  0.899085 8.268275 	l:9.772867 	em:0.250000

Epoch performance:  8.5027sec Trl:9.849642 	Trem:0.166667 	Teem:0.000000

Epoch:  54 / 300
Batch:	0 /9	:  1.033774 1.033794 	l:9.542910 	em:0.250000
Batch:	1 /9	:  0.894207 1.929109 	l:9.863150 	em:0.150000
Batch:	2 /9	:  0.893630 2.824171 	l:9.766682 	em:0.200000
Batch:	3 /9	:  0.915370 3.740462 	l:9.776108 	em:0.200000
Batch:	4 /9	:  0.908362 4.649950 	l:9.771021 	em:0.200000
Batch:	5 /9	:  0.876154 5.528662 	l:9.921696 	em:0.250000
Batch:	6 /9	:  0.898427 6.428042 	l:

...

Batch:	8 /9	:  0.898456 8.910141 	l:9.864877 	em:0.100000

Epoch performance:  9.1185sec Trl:9.777438 	Trem:0.183333 	Teem:0.000000

Epoch:  80 / 300
Batch:	0 /9	:  1.006324 1.006343 	l:10.062418 	em:0.100000
Batch:	1 /9	:  1.051811 2.058564 	l:9.762848 	em:0.300000
Batch:	2 /9	:  1.085601 3.144469 	l:9.862690 	em:0.150000
Batch:	3 /9	:  0.892783 4.039111 	l:9.663502 	em:0.100000
Batch:	4 /9	:  0.942768 4.982861 	l:9.863153 	em:0.250000
Batch:	5 /9	:  0.886471 5.870352 	l:9.811838 	em:0.100000
Batch:	6 /9	:  0.943176 6.814800 	l:10.012620 	em:0.200000
Batch:	7 /9	:  0.828727 7.644549 	l:9.762413 	em:0.250000
Batch:	8 /9	:  0.953428 8.599100 	l:9.613988 	em:0.200000

Epoch performance:  8.7743sec Trl:9.823941 	Trem:0.183333 	Teem:0.000000

Epoch:  81 / 300
Batch:	0 /9	:  0.815773 0.815798 	l:9.826317 	em:0.150000
Batch:	1 /9	:  0.957185 1.774363 	l:9.664312 	em:0.150000
Batch:	2 /9	:  1.037051 2.812227 	l:9.863078 	em:0.100000
Batch:	3 /9	:  0.920560 3.734261 	l:9.563349 	em:0.300000
Ba

Batch:	1 /9	:  0.917412 1.915112 	l:9.613243 	em:0.250000
Batch:	2 /9	:  0.945040 2.861433 	l:9.513802 	em:0.350000
Batch:	3 /9	:  1.075495 3.938235 	l:9.763649 	em:0.150000
Batch:	4 /9	:  0.828039 4.767361 	l:9.662563 	em:0.350000
Batch:	5 /9	:  0.855439 5.623368 	l:9.613750 	em:0.250000
Batch:	6 /9	:  0.962767 6.587131 	l:9.913202 	em:0.100000
Batch:	7 /9	:  0.944153 7.532160 	l:9.863078 	em:0.150000
Batch:	8 /9	:  0.822201 8.355576 	l:9.413723 	em:0.350000

Epoch performance:  8.5612sec Trl:9.685609 	Trem:0.233333 	Teem:0.000000

Epoch:  94 / 300
Batch:	0 /9	:  1.046506 1.046526 	l:9.716511 	em:0.250000
Batch:	1 /9	:  0.962167 2.009742 	l:9.862833 	em:0.250000
Batch:	2 /9	:  0.894515 2.904850 	l:9.962805 	em:0.100000
Batch:	3 /9	:  0.918184 3.824048 	l:9.513551 	em:0.300000
Batch:	4 /9	:  1.143971 4.968771 	l:9.862774 	em:0.150000
Batch:	5 /9	:  1.184428 6.154007 	l:9.713318 	em:0.200000
Batch:	6 /9	:  1.018018 7.173410 	l:9.313643 	em:0.400000
Batch:	7 /9	:  0.912127 8.086539 	l:9.

Batch:	5 /9	:  0.874773 5.890147 	l:9.862997 	em:0.100000
Batch:	6 /9	:  0.913326 6.804672 	l:9.762305 	em:0.200000
Batch:	7 /9	:  0.980067 7.785429 	l:9.763287 	em:0.100000
Batch:	8 /9	:  1.135389 8.921422 	l:9.763074 	em:0.250000

Epoch performance:  9.1292sec Trl:9.724234 	Trem:0.183333 	Teem:0.000000

Epoch:  107 / 300
Batch:	0 /9	:  1.004570 1.004589 	l:9.762892 	em:0.250000
Batch:	1 /9	:  0.927675 1.933098 	l:9.762607 	em:0.200000
Batch:	2 /9	:  0.985599 2.919764 	l:9.712886 	em:0.200000
Batch:	3 /9	:  0.870497 3.791372 	l:9.562945 	em:0.300000
Batch:	4 /9	:  0.838988 4.630970 	l:9.812641 	em:0.200000
Batch:	5 /9	:  0.964239 5.595808 	l:9.813173 	em:0.100000
Batch:	6 /9	:  1.176316 6.773092 	l:9.413108 	em:0.350000
Batch:	7 /9	:  1.018527 7.793040 	l:9.912504 	em:0.050000
Batch:	8 /9	:  0.939591 8.733213 	l:9.613043 	em:0.250000

Epoch performance:  8.9136sec Trl:9.707311 	Trem:0.211111 	Teem:0.000000

Epoch:  108 / 300
Batch:	0 /9	:  0.816971 0.816991 	l:9.812981 	em:0.100000
Ba


Epoch performance:  8.4890sec Trl:9.585461 	Trem:0.283333 	Teem:0.000000

Epoch:  120 / 300
Batch:	0 /9	:  1.014940 1.014958 	l:9.463761 	em:0.400000
Batch:	1 /9	:  0.984979 2.000553 	l:9.812428 	em:0.200000
Batch:	2 /9	:  0.855098 2.856903 	l:9.663096 	em:0.100000
Batch:	3 /9	:  0.854748 3.712774 	l:9.463186 	em:0.350000
Batch:	4 /9	:  0.869599 4.582649 	l:9.762732 	em:0.250000
Batch:	5 /9	:  0.954479 5.537350 	l:9.463358 	em:0.350000
Batch:	6 /9	:  0.865898 6.404870 	l:9.763111 	em:0.150000
Batch:	7 /9	:  0.860257 7.266157 	l:9.613073 	em:0.300000
Batch:	8 /9	:  0.863730 8.130424 	l:9.563135 	em:0.300000

Epoch performance:  8.3691sec Trl:9.618653 	Trem:0.266667 	Teem:0.000000

Epoch:  121 / 300
Batch:	0 /9	:  1.015220 1.015244 	l:9.763172 	em:0.150000
Batch:	1 /9	:  0.814673 1.831068 	l:9.363751 	em:0.350000
Batch:	2 /9	:  0.875295 2.706912 	l:9.413211 	em:0.450000
Batch:	3 /9	:  0.959120 3.666738 	l:9.563142 	em:0.250000
Batch:	4 /9	:  0.934257 4.602485 	l:9.313277 	em:0.500000
Ba

Batch:	2 /9	:  1.073647 2.955473 	l:9.513172 	em:0.350000
Batch:	3 /9	:  0.872341 3.828992 	l:9.613461 	em:0.150000
Batch:	4 /9	:  0.849138 4.678964 	l:9.713265 	em:0.250000
Batch:	5 /9	:  1.103000 5.783164 	l:9.662983 	em:0.300000
Batch:	6 /9	:  1.066426 6.850890 	l:9.812792 	em:0.100000
Batch:	7 /9	:  0.979954 7.832031 	l:9.662844 	em:0.200000
Batch:	8 /9	:  0.893759 8.726369 	l:9.563050 	em:0.300000

Epoch performance:  8.9259sec Trl:9.651927 	Trem:0.238889 	Teem:0.005102

Epoch:  134 / 300
Batch:	0 /9	:  0.845987 0.846007 	l:9.513051 	em:0.350000
Batch:	1 /9	:  1.040123 1.887426 	l:9.313068 	em:0.500000
Batch:	2 /9	:  0.893584 2.781278 	l:9.662793 	em:0.250000
Batch:	3 /9	:  1.006500 3.788455 	l:9.712775 	em:0.250000
Batch:	4 /9	:  0.963308 4.752091 	l:9.712475 	em:0.200000
Batch:	5 /9	:  0.842173 5.594843 	l:9.862380 	em:0.200000
Batch:	6 /9	:  0.842191 6.438354 	l:9.663465 	em:0.150000
Batch:	7 /9	:  0.851934 7.291454 	l:9.613250 	em:0.250000
Batch:	8 /9	:  0.921447 8.213878 	l:9

In [ ]:
# See if gradients are being passed
p = list(filter(lambda p: p.requires_grad, ques_model.parameters())) + \
    list(filter(lambda p: p.requires_grad, para_model.parameters())) + \
    list(para_model.parameters()) + \
    list(pointer_decoder_model.parameters())
                       
print([ x.grad.sum().item() for x in p])                     

#### Visualizations

So far, we plot the training losss. 
Shall we superimpose test loss on it too? We don't calculate test loss per batch though (fortunately).

In [ ]:
# Visualizations
print("Training Loss")
visualize_loss(op[0], "train loss", _only_epoch=False)

# if len(op[1]) > 0:

print("Training EM")
visualize_loss(op[1], "train em")

print("Testing EM")
visualize_loss(op[3], "test em")

